### Preprocessing

In [182]:
import pandas as pd
from gensim.models import Word2Vec
import string
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adityaanand/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [183]:
path = "../../resources/IndianFoodDatasetCSV.csv"
df = pd.read_csv(path)
sampled_data = df
df.head()

,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
0,1,Masala Karela Recipe,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,Salt - to taste,1 Onion - thinly sliced,3 tablespoon Gram flour (besan),2 teaspoons Turmeric powder (Haldi),1 tablespoon Red Chilli powder,2 teaspoons Cumin seeds (Jeera),1 tablespoon Coriander Powder (Dhania),1 tablespoon Amchur (Dry Mango Powder),Sunflower Oil - as required","6 Karela (Bitter Gourd/ Pavakkai) - deseeded,Salt - to taste,1 Onion - thinly sliced,3 tablespoon Gram flour (besan),2 teaspoons Turmeric powder (Haldi),1 tablespoon Red Chilli powder,2 teaspoons Cumin seeds (Jeera),1 tablespoon Coriander Powder (Dhania),1 tablespoon Amchur (Dry Mango Powder),Sunflower Oil - as required",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-seed the karela and slice. Do not remove the skin as the skin has all the nutrients. Add the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles. Release the pressure immediately and open the lids. Keep aside.Heat oil in a heavy bottomed pan or a kadhai. Add cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan. Stir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again. Cover the pan and simmer Masala Karela stirring occasionally until everything comes together well. Turn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family. \n","To begin making the Masala Karela Recipe,de-seed the karela and slice. Do not remove the skin as the skin has all the nutrients. Add the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles. Release the pressure immediately and open the lids. Keep aside.Heat oil in a heavy bottomed pan or a kadhai. Add cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan. Stir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again. Cover the pan and simmer Masala Karela stirring occasionally until everything comes together well. Turn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family.",https://www.archanaskitchen.com/masala-karela-recipe
1,2,टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Recipe In Hindi),Spicy Tomato Rice (Recipe),"2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बीसी बेले भात पाउडर,नमक - स्वाद अनुसार,1 छोटा चमच्च चना दाल,1/2 छोटा चमच्च जीरा,1 छोटा चमच्च सफ़ेद उरद दाल,1/2 छोटा चमच्च राइ,1 हरी मिर्च,1 सुखी लाल मिर्च,2 छोटा चमच्च काजू - या मूंगफली,1-1/2 बड़े चमच्च तेल - प्रयोग अनुसार,1/2 छोटा चमच्च हींग","2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teaspoons BC Belle Bhat powder, salt - as per taste, 1 teaspoon chickpea lentils, 1/2 teaspoon cumin seeds, 1 teaspoon white urad dal, 1/2 Teaspoon mustard, 1 green chilli, 1 dry red chilli, 2 teaspoon cashew - or peanuts, 1-1 / 2 tablespoon oil - 1/2 teaspoon asafoetida",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"टमाटर पुलियोगरे बनाने के लिए सबसे पहले टमाटर को काट ले. अब एक मिक्सर ग्राइंडर में डाले और उसकी प्यूरी बना ले. अब एक कढ़ाई में तेल गरम करें। तेल के गरम होने के बाद इसमें चना दाल, उरद दाल, काजू डाले और 10 से 20 सेकण्ड्स तक पकने दे. 10 से 20 सेकण्ड्स के बाद इसमें जीरा, राइ, हरी मिर्च, सुखी लाल मिर्च और कढ़ी पत्ता डाले। 30 सेकण्ड्स बाद इसमें टमाटर की प्यूरी डाले और मिला ले. इसमें बीसी बेले भात पा

In [184]:
df.columns

Index(['Srno', 'RecipeName', 'TranslatedRecipeName', 'Ingredients',
       'TranslatedIngredients', 'PrepTimeInMins', 'CookTimeInMins',
       'TotalTimeInMins', 'Servings', 'Cuisine', 'Course', 'Diet',
       'Instructions', 'TranslatedInstructions', 'URL'],
      dtype='object')

In [185]:
df.shape

(6871, 15)

Null Values check

In [186]:
df.isnull().sum()

Srno                      0
RecipeName                0
TranslatedRecipeName      0
Ingredients               6
TranslatedIngredients     6
PrepTimeInMins            0
CookTimeInMins            0
TotalTimeInMins           0
Servings                  0
Cuisine                   0
Course                    0
Diet                      0
Instructions              0
TranslatedInstructions    0
URL                       0
dtype: int64

In [187]:
df[df['Ingredients'].isnull()][:1]

,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
287,306,Pear And Walnut Salad Recipe,Pear And Walnut Salad Recipe,NaN,NaN,10,30,40,2,Continental,Appetizer,Vegetarian,"To begin making the Pear And Walnut Salad Recipe, first dry roast the walnuts in a pan or skillet for few seconds. Once the walnuts have become lightly crisp, turn off the heat and keep it aside.Tear the lettuce leaves into bite size pieces and keep aside.To prepare the dressing for the salad, in a small bowl whisk together olive oil, lemon juice, dijon mustard, tarragon leaves, salt and red chili.Next in a salad mixing bowl add the lettuce, diced cucumber, sliced pear. Pour the dressing and gently toss the salad in the dressing. Season with salt and pepper to taste. Finally toasted walnuts to the salad. Gently toss the salad. Check the salt and seasonings and adjust to suit your taste.Serve Pear And Walnut Salad for the light weekday lunch or the summer healthy supper along with Spinach And Soya Sub (Sandwich) immediately after tossing in the dressing. If the salad is not consumed immediately the lettuce leaves and the walnuts will become soggy.\n","To begin making the Pear And Walnut Salad Recipe, first dry roast the walnuts in a pan or skillet for few seconds. Once the walnuts have become lightly crisp, turn off the heat and keep it aside.Tear the lettuce leaves into bite size pieces and keep aside.To prepare the dressing for the salad, in a small bowl whisk together olive oil, lemon juice, dijon mustard, tarragon leaves, salt and red chili.Next in a salad mixing bowl add the lettuce, diced cucumber, sliced pear. Pour the dressing and gently toss the salad in the dressing. Season with salt and pepper to taste. Finally toasted walnuts to the salad. Gently toss the salad. Check the salt and seasonings and adjust to suit your taste.Serve Pear And Walnut Salad for the light weekday lunch or the summer healthy supper along with Spinach And Soya Sub (Sandwich) immediately after tossing in the dressing. If the salad is not consumed immediately the lettuce leaves and the walnuts will become soggy.",https://www.archanaskitchen.com/pear-and-walnut-salad-recipe


In [188]:
pd.set_option('display.max_colwidth',None)
df[df['Ingredients'].isnull()][['RecipeName','Ingredients','URL']]

,RecipeName,Ingredients,URL
287,Pear And Walnut Salad Recipe,NaN,https://www.archanaskitchen.com/pear-and-walnut-salad-recipe
1262,Spinach and Cottage Cheese Eggless Ravioli Recipe,NaN,http://www.archanaskitchen.com/spinach-and-cottage-cheese-eggless-ravioli-recipe
1809,Thai Jasmine Sticky Rice Recipe,NaN,https://www.archanaskitchen.com/thai-jasmine-sticky-rice-recipe
1827,Classic Pavakkai Stir Fry Recipe (Bitter Gourd Fry),NaN,https://www.archanaskitchen.com/classic-pavakkai-stir-fry-recipe-bitter-gourd-fry
5386,Urulaikizhangu Puli Thokku Recipe (South Indian Style Potatoes with Tamarind),NaN,https://www.archanaskitchen.com/urulaikizhangu-puli-thokku-recipe-south-indian-style-potatoes-with-tamarind
5586,Mashed Peas Recipe,NaN,http://www.archanaskitchen.com/mashed-peas-recipe


drop NA

In [189]:
df.dropna(inplace=True)

In [190]:
df.isnull().sum()

Srno                      0
RecipeName                0
TranslatedRecipeName      0
Ingredients               0
TranslatedIngredients     0
PrepTimeInMins            0
CookTimeInMins            0
TotalTimeInMins           0
Servings                  0
Cuisine                   0
Course                    0
Diet                      0
Instructions              0
TranslatedInstructions    0
URL                       0
dtype: int64

In [191]:
df.shape

(6865, 15)

In [192]:
df.size

102975

In [193]:
df.isna().sum()

Srno                      0
RecipeName                0
TranslatedRecipeName      0
Ingredients               0
TranslatedIngredients     0
PrepTimeInMins            0
CookTimeInMins            0
TotalTimeInMins           0
Servings                  0
Cuisine                   0
Course                    0
Diet                      0
Instructions              0
TranslatedInstructions    0
URL                       0
dtype: int64

In [194]:
df.isnull().sum()

Srno                      0
RecipeName                0
TranslatedRecipeName      0
Ingredients               0
TranslatedIngredients     0
PrepTimeInMins            0
CookTimeInMins            0
TotalTimeInMins           0
Servings                  0
Cuisine                   0
Course                    0
Diet                      0
Instructions              0
TranslatedInstructions    0
URL                       0
dtype: int64

In [195]:
pd.reset_option('display.max_colwidth')

In [196]:
df.head()


,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
0,1,Masala Karela Recipe,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...","6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...","To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...
1,2,टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...,Spicy Tomato Rice (Recipe),"2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...","2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,टमाटर पुलियोगरे बनाने के लिए सबसे पहले टमाटर क...,"To make tomato puliogere, first cut the tomato...",http://www.archanaskitchen.com/spicy-tomato-ri...
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...","1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...","To begin making the Ragi Vermicelli Recipe, fi...",http://www.archanaskitchen.com/ragi-vermicelli...
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...","500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,To begin making Gongura Chicken Curry Recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,5,आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 बड़ा चमच्च चना दाल,1 बड़ा चमच्च सफ़ेद उरद दाल,2...","1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,आंध्रा स्टाइल आलम पचड़ी बनाने के लिए सबसे पहले ...,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...


In [197]:
df.duplicated().sum()

0

In [198]:
df.dtypes

Srno                       int64
RecipeName                object
TranslatedRecipeName      object
Ingredients               object
TranslatedIngredients     object
PrepTimeInMins             int64
CookTimeInMins             int64
TotalTimeInMins            int64
Servings                   int64
Cuisine                   object
Course                    object
Diet                      object
Instructions              object
TranslatedInstructions    object
URL                       object
dtype: object

In [199]:
df['Cuisine'].unique()

array(['Indian', 'South Indian Recipes', 'Andhra', 'Udupi', 'Mexican',
       'Fusion', 'Continental', 'Bengali Recipes', 'Punjabi', 'Chettinad',
       'Tamil Nadu', 'Maharashtrian Recipes', 'North Indian Recipes',
       'Italian Recipes', 'Sindhi', 'Thai', 'Chinese', 'Kerala Recipes',
       'Gujarati Recipes\ufeff', 'Coorg', 'Rajasthani', 'Asian',
       'Middle Eastern', 'Coastal Karnataka', 'European', 'Kashmiri',
       'Karnataka', 'Lucknowi', 'Hyderabadi', 'Side Dish', 'Goan Recipes',
       'Arab', 'Assamese', 'Bihari', 'Malabar', 'Himachal', 'Awadhi',
       'Cantonese', 'North East India Recipes', 'Sichuan', 'Mughlai',
       'Japanese', 'Mangalorean', 'Vietnamese', 'British',
       'North Karnataka', 'Parsi Recipes', 'Greek', 'Nepalese',
       'Oriya Recipes', 'French', 'Indo Chinese', 'Konkan',
       'Mediterranean', 'Sri Lankan', 'Haryana', 'Uttar Pradesh',
       'Malvani', 'Indonesian', 'African', 'Shandong', 'Korean',
       'American', 'Kongunadu', 'Pakistani', 'C

In [200]:
pd.set_option('display.max_colwidth',None)
df['TranslatedRecipeName'] = df['TranslatedRecipeName'].str.strip()
df['TranslatedRecipeName'].head()

0                                           Masala Karela Recipe
1                                     Spicy Tomato Rice (Recipe)
2     Ragi Semiya Upma Recipe - Ragi Millet Vermicelli Breakfast
3    Gongura Chicken Curry Recipe - Andhra Style Gongura Chicken
4      Andhra Style Alam Pachadi Recipe - Adrak Chutney (Recipe)
Name: TranslatedRecipeName, dtype: object

Removing the word 'Recipe' from TranslatedRecipeName to make it more general

In [201]:
df['TranslatedRecipeName'][df['TranslatedRecipeName'].str.contains('\(Recipe\) | Recipe')].size

6353

In [202]:
df['TranslatedRecipeName'] = df['TranslatedRecipeName'].str.replace(r'\(Recipe\)|Recipe', '', regex=True)

In [203]:
df['TranslatedRecipeName'][df['TranslatedRecipeName'].str.contains('Reciper')].size

0

In [204]:
df['TranslatedRecipeName'].head()

0                                           Masala Karela 
1                                       Spicy Tomato Rice 
2     Ragi Semiya Upma  - Ragi Millet Vermicelli Breakfast
3    Gongura Chicken Curry  - Andhra Style Gongura Chicken
4              Andhra Style Alam Pachadi  - Adrak Chutney 
Name: TranslatedRecipeName, dtype: object

#### Creating Embedding

In [205]:
dataToEmbed = df['TranslatedRecipeName']
dataToEmbed = [data.split() for data in df['TranslatedRecipeName']]

Each Input to the model must be a list of string.

In [206]:
w2v = Word2Vec(dataToEmbed,vector_size=100,window=5,workers=5,epochs=10,min_count=5)

In [207]:
w2v.wv.most_similar('Masala')

[('Bhindi', 0.9943283200263977),
 ('Gobi', 0.9924203157424927),
 ('Arbi', 0.9914670586585999),
 ('Achari', 0.9909719824790955),
 ('Baingan', 0.9908280968666077),
 ('Ka', 0.9906718134880066),
 ('Parwal', 0.9903978109359741),
 ('Tamatar', 0.9903918504714966),
 ('Besan', 0.9897860288619995),
 ('Makhani', 0.9893636107444763)]

In [ ]:
words = ['Masala', 'Karela']  # Replace with your list of words
similar_words = w2v.wv.most_similar(positive=words, topn=10)

# Example: print top 10 similar words
for word, similarity in similar_words:
    print(f"Word: {word}, Similarity: {similarity}")

Word: Achari, Similarity: 0.9971821308135986
Word: Besan, Similarity: 0.9966304898262024
Word: Dum, Similarity: 0.9966135621070862
Word: Mirch, Similarity: 0.9962526559829712
Word: Ka, Similarity: 0.9959337115287781
Word: Chole, Similarity: 0.99583500623703
Word: Wali, Similarity: 0.9957400560379028
Word: Wale, Similarity: 0.9955218434333801
Word: Bharta, Similarity: 0.9953157901763916
Word: Arbi, Similarity: 0.9952240586280823


In [209]:
w2v.wv.most_similar('Karela')

[('Chole', 0.9972859621047974),
 ('Bihari', 0.9972686767578125),
 ('Saag', 0.9967897534370422),
 ('Mirch', 0.9967010021209717),
 ('Amritsari', 0.9964016079902649),
 ('Mooli', 0.9962871670722961),
 ('Khatti', 0.9962379336357117),
 ('Papad', 0.9961793422698975),
 ('Bharta', 0.9960885643959045),
 ('Bharwa', 0.9960362315177917)]

In [210]:
df[['RecipeName', 'Cuisine']].astype(str).agg(' '.join, axis=1)

0                                                                              Masala Karela Recipe Indian
1                        टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Recipe In Hindi) South Indian Recipes
2                          Ragi Semiya Upma Recipe - Ragi Millet Vermicelli Breakfast South Indian Recipes
3                                       Gongura Chicken Curry Recipe - Andhra Style Gongura Chicken Andhra
4                                   आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney (Recipe In Hindi) Andhra
                                                       ...                                                
6866                                    गोअन मशरुम जकुटी रेसिपी - Goan Mushroom Xacuti Recipe Goan Recipes
6867    शकरकंदी और मेथी का पराठा रेसिपी - Sweet Potato & Methi Stuffed Paratha Recipe North Indian Recipes
6868                                                  Ullikadala Pulusu Recipe | Spring Onion Curry Andhra
6869                                 

In [211]:
df.columns

Index(['Srno', 'RecipeName', 'TranslatedRecipeName', 'Ingredients',
       'TranslatedIngredients', 'PrepTimeInMins', 'CookTimeInMins',
       'TotalTimeInMins', 'Servings', 'Cuisine', 'Course', 'Diet',
       'Instructions', 'TranslatedInstructions', 'URL'],
      dtype='object')

In [212]:
stemmer_instance = nltk.stem.PorterStemmer()
stop_words = stopwords.words('english')

def tokenize_recipe(text):
    text = str(text)
    for punctuation_char in string.punctuation:
        text = text.replace(punctuation_char, '').lower()
    
    words = text.split(' ')
    stemmed_words = []
    
    for word in words:
        if (word not in stop_words) and (word != ''):
            stemmed_words.append(stemmer_instance.stem(word))
    
    return stemmed_words

def generate_word_embeddings(dataframe, text_column):
    tokenized_text = dataframe[text_column].apply(tokenize_recipe)
    word2vec_model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
    word_vectors = {key: word2vec_model.wv[key] for key in word2vec_model.wv.index_to_key}
    
    return word_vectors

In [213]:
def compute_and_store_embeddings(data_frame):
    ingredient_embeddings_model = generate_word_embeddings(data_frame, 'TranslatedIngredients')
    data_frame['combined_text'] = data_frame[['TranslatedRecipeName', 'Diet', 'TranslatedInstructions']].astype(str).agg(' '.join, axis=1)
    data_frame['combined_text'] = data_frame['combined_text'].str.lower()

    tfidf_model = TfidfVectorizer(min_df=5, tokenizer=tokenize_recipe)
    vectorized_text = tfidf_model.fit_transform(data_frame['combined_text'])

    ingredient_vectors = [
        np.mean([ingredient_embeddings_model[word] for word in tokenize_recipe(ingredient_list) if word in ingredient_embeddings_model] 
                or [np.zeros(100)], axis=0) 
        for ingredient_list in data_frame['TranslatedIngredients']
    ]

    final_embeddings = np.concatenate([vectorized_text.toarray(), np.array(ingredient_vectors)], axis=1)

    with open('final_embeddings.pkl', 'wb') as embedding_file:
        pickle.dump(final_embeddings, embedding_file)
    
    with open('tfidf_model.pkl', 'wb') as tfidf_file:
        pickle.dump(tfidf_model, tfidf_file)
    
    print("Embeddings and TF-IDF vectorizer successfully saved!")
    

In [214]:
embeddings = generate_word_embeddings(sampled_data, 'TranslatedIngredients')

In [215]:
sampled_data['combined_text'] = sampled_data[['TranslatedRecipeName', 'Diet', 'TranslatedInstructions']].astype(str).agg(' '.join, axis=1)


sampled_data['combined_text'] = sampled_data['combined_text'].str.lower()

tfidf_vectorizer = TfidfVectorizer(min_df=5, tokenizer=tokenize_recipe)
vectorized_text_data = tfidf_vectorizer.fit_transform(sampled_data['combined_text'])

/Users/adityaanand/Desktop/Desktop/DataScience/env/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [216]:
vectorized_text_data.toarray().shape


(6865, 5721)

In [220]:

def load_combined_data_and_model():
    with open('final_embeddings.pkl', 'rb') as file:
        combined_data = pickle.load(file)
    with open('tfidf_model.pkl', 'rb') as file:
        tfidf_model = pickle.load(file)
    return combined_data, tfidf_model

In [225]:
def search_similar_recipes(data_frame, user_query, top_n=5):
    # try:
    #     combined_data, tfidf_model = load_combined_data_and_model()
    # except FileNotFoundError:
    compute_and_store_embeddings(data_frame)
    combined_data, tfidf_model = load_combined_data_and_model()

    query_frame = pd.DataFrame({'combined_text': [user_query]})
    query_frame['combined_text'] = query_frame['combined_text'].str.lower()
    query_vectorized = tfidf_model.transform(query_frame['combined_text'])

    feature_gap = combined_data.shape[1] - query_vectorized.shape[1]
    if feature_gap > 0:
        query_vectorized = np.pad(query_vectorized.toarray(), ((0, 0), (0, feature_gap)))

    similarity_scores = cosine_similarity(query_vectorized, combined_data)
    top_matches = similarity_scores[0].argsort()[::-1][:top_n]
    matched_recipe_names = data_frame.iloc[top_matches]['TranslatedRecipeName'].tolist()

    return matched_recipe_names

In [231]:
# Test
similiar_receipe = search_similar_recipes(sampled_data, "spicy chicken in chinese style")
similiar_receipe

/Users/adityaanand/Desktop/Desktop/DataScience/env/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Embeddings and TF-IDF vectorizer successfully saved!


['Spicy Chicken Curry  In Naga Style',
 'Chinese Fried Rice  With Chicken And Shredded Lettuce',
 'Chinese Dragon Chicken With Vegetables ',
 'Chinese Chicken Fried Rice ',
 'Pan Fried Chinese Chilli Chicken recipe']

In [232]:
sampled_data.loc[sampled_data['TranslatedRecipeName'].isin(similiar_receipe), ['TranslatedRecipeName', 'Diet','Cuisine','TranslatedInstructions','CookTimeInMins']]

,TranslatedRecipeName,Diet,Cuisine,TranslatedInstructions,CookTimeInMins
1713,Chinese Chicken Fried Rice,Non Vegeterian,Chinese,"To make Chinese Chicken Fried Rice, first heat the oil in a pan. Add chicken pieces to it and cook it well. After the chicken is cooked, add soy sauce, salt, pepper powder and cook it for a few seconds. Now add lettuce, mix and let it cook for 4 to 5 minutes. After 4 to 5 minutes, add cooked rice and mix it well. After cooking for 2 minutes, turn off the gas and serve. Serve Chinese Chicken Fried Rice Recipe with Soya Chunks Manchurian for dinner.",10
1756,Pan Fried Chinese Chilli Chicken recipe,Non Vegeterian,Chinese,"To prepare Pan Fried Chinese Chilli Chicken recipe, wash the chicken breasts well with water and then cut them into vertical strips or small cubes.In a bowl, beat the egg and combine with soy sauce, black pepper powder, red chili powder, rice flour and the chicken strips, mix well.Heat a Skillet and drizzle oil.Place all the coated chicken pieces, well apart from each other.Let them fry on medium high heat till nice and soft on all the sides and are golden brown.Check with the toothpick if the chicken is cooked well.While chicken is cooking, prepare the sauce. In a mixing bowl, combine the soy sauce, red chili sauce, tomato ketchup, crushed black pepper and honey. Heat a tablespoon oil in a saucepan and add chopped garlic. Saute for few seconds and then add chilies, onions and bell pepper. Mix and cover. Let it cook on medium heat for 2-3 minutes.Pour in the prepared sauce and mix.Add the pan fried chicken and mix well.Sprinkle some chopped spring onions and mix.Simmer on low heat for 5 minutes and remove from heat.Serve this delicious Pan Fried Chinese Chilli Chicken as an appetizer or make a meal with plain rice/Chinese Vegetable Fried Rice and Black Sesame Nai Huang Bao Recipe.",35
1915,Chinese Fried Rice With Chicken And Shredded Lettuce,Non Vegeterian,Chinese,"To begin making Chinese Fried Rice Recipe With Chicken And Shredded Lettuce, heat oil in a chinese wok and add chicken pieces to it. Saute till chicken is soft and cooked.Now add soy sauce, salt, black pepper powder and saute for another few seconds.Add shredded lettuce and mix well and fry for 4-5 mins till lettuce is partially tender.Take 2 cups of cooked rice and add in the chicken mix. Toss for couple of minutes till everything incorporates well.Turn off the gas and transfer to a platter and serve. Serve Chinese Fried Rice Recipe With Chicken And Shredded Lettuce with Soy Chunks Manchurian Recipe and Chinese Darsaan Recipe With Ice Cream for dessert.\n",10
3900,Chinese Dragon Chicken With Vegetables,Non Vegeterian,Chinese,"To begin with Chinese Dragon Chicken With Vegetables Recipe, get prep with all the ingredients first. Add 1.5 tablespoons of ginger garlic paste to chicken and mix well. Leave it to marinate for an hour. (save the rest of the ginger garlic paste for later use).Add beaten egg to it along with corn flour, salt, pepper and red chili flakes. Set it aside.Heat some oil in a wok, saute all the vegetables - babycorn, red, yellow and green peppers, carrots, onions and mushrooms until tender but crunchy. Set it aside on a plate.In a shallow fry pan, shallow fry all the chicken pieces till cooked but crisp. You can alternately, deep fry the marinated chicken pieces, till crunchy.In the wok, heat about 2 teaspoons of oil. Add red chillies, remaining minced ginger garlic, sauté it well, till the raw smell disappears.Add the fried chicken pieces, sauteed vegetables and mushroom.Add Sichuan sauce, tomato ketchup, honey, salt. Add more pepper if you prefer. Mix well.Adjust with a little water if required.Saute for a couple of minutes till combined.Serve Chinese Dragon Chicken With Vegetables Recipe along with and Bao buns or Chinese Vegetable Fried Rice and end it with a Toffee Banana Recipe (Classic Chinese Dessert).\n",30
5996,Spicy Chicken Curry In Naga Style,High Protein Non Vegetarian,Nagaland,"To begin making S

In [233]:
sampled_data.to_pickle('dataframe.pkl')